In [1]:
import requests
import pandas as pd
import numpy as np
import pprint
import matplotlib.pyplot as plt
pp = pprint.PrettyPrinter(indent=4)

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()
matches_url = 'https://api.football-data.org/v2/competitions/PL/matches'
teams_url = 'https://api.football-data.org/v2/competitions/PL/teams'
fixtures_response = requests.get(matches_url,headers={'X-Auth-Token': '3344f6dea3c44934a3a7c7eb185352de'})
teams_response = requests.get(teams_url, headers={'X-Auth-Token': '3344f6dea3c44934a3a7c7eb185352de'} )

In [3]:
matches_json = fixtures_response.json()
teams_json = teams_response.json()
teams_df = pd.DataFrame(json['teams'])

In [30]:
team_strength = dict()
team_attack_strength_home = dict()
team_attack_strength_away = dict()
team_defence_strength_home = dict()
team_defence_strength_away = dict()

for i, team in enumerate(teams_df["name"]):
    team_strength[team] = teams_df["strength"][i]
    team_attack_strength_home[team] = teams_df["strength_attack_home"][i]
    team_attack_strength_away[team] = teams_df["strength_attack_away"][i]
    team_defence_strength_home[team] = teams_df["strength_defence_home"][i]
    team_defence_strength_away[team] = teams_df["strength_defence_away"][i]



In [107]:
team_defence_strength_home

{'Arsenal': 1220,
 'Aston Villa': 1080,
 'Brentford': 1020,
 'Brighton': 1100,
 'Burnley': 1040,
 'Chelsea': 1250,
 'Crystal Palace': 1010,
 'Everton': 1200,
 'Leicester': 1150,
 'Leeds': 1210,
 'Liverpool': 1300,
 'Man City': 1340,
 'Man Utd': 1240,
 'Newcastle': 1030,
 'Norwich': 1020,
 'Southampton': 1110,
 'Spurs': 1250,
 'Watford': 1010,
 'West Ham': 1260,
 'Wolves': 1040}

In [28]:
def api_name_to_fpl(name):
    outliers = { 
        "Wolverhampton Wanderers FC":"Wolves",
        "Manchester United FC":"Man Utd",
        "Manchester City FC": "Man City",
        "Tottenham Hotspur FC": "Spurs",
        "Sheffield United FC":"Sheffield Utd"
        
    }
    for i in teams_df["name"]:
        if(i in name):
            return i
        
    return outliers[name]

def get_team_strength(name):
    return team_strength[api_name_to_fpl(name)]

def get_team_attacking_strength(name, home_or_away):
    if home_or_away == 'a':
        return team_attack_strength_away[api_name_to_fpl(name)]
    elif home_or_away == 'h':
        return team_attack_strength_home[api_name_to_fpl(name)]
    else:
        raise Exception("neither h or a provided")

def calculate_opposition_attacking_strength_fixtures(list_games):
    difficulty_tuples=[]
    for i in list_games:
        difficulty_tuples.append((i[0],get_team_attacking_strength(i[0], i[1])))
    return difficulty_tuples

def get_team_defensive_strength(name, home_or_away):
    if home_or_away == 'a':
        return team_defence_strength_home[api_name_to_fpl(name)]
    elif home_or_away == 'h':
        return team_defence_strength_away[api_name_to_fpl(name)]
    else:
        raise Exception("neither h or a provided")

def calculate_opposition_defensive_strength_fixtures(list_games):
    difficulty_tuples=[]
    for i in list_games:
        difficulty_tuples.append((i[0],get_team_defensive_strength(i[0], i[1])))
    return difficulty_tuples

In [19]:
fixture_dict = dict()

for i in teams_json["teams"]:
    opposition = []
    for j in matches_json["matches"]:
        if(i["name"]==j["homeTeam"]["name"]):
            opposition.append((j["awayTeam"]["name"], "h"))
        elif(i["name"]==j["awayTeam"]["name"]):
            opposition.append((j["homeTeam"]["name"], "a"))
    fixture_dict[i["name"]] = opposition

api_team_names = []
for i in fixture_dict:
    api_team_names.append(i)

# Opposition attacking strength (used for picking defenders)

In [112]:
def generate_attacking_strength_fixtures(starting_gw, rolling):
    opposition_attacking_strength_fixtures = {}
    for i in api_team_names:
        for j in fixture_dict[i]:
            opposition_attacking_strength_fixtures[i] = calculate_opposition_attacking_strength_fixtures(fixture_dict[i])
    rolling_strength = {"team_name": [], "rolling_attacking_strength": []}
    for team_name, fixtures in opposition_attacking_strength_fixtures.items():
        rolling_strength["team_name"].append(team_name)
        fixtures_to_consider = fixtures[starting_gw-1:starting_gw-1+gws_to_roll]
        rolling_strength["rolling_attacking_strength"].append(int(np.mean([item[1] for item in fixtures_to_consider])))
    rolling_attacking_strength_df = pd.DataFrame(rolling_strength)
    rolling_attacking_strength_df.set_index("team_name", inplace=True)
    return rolling_attacking_strength_df

## Opposition defensive strength (used for picking attackers and midfielders)

In [109]:
def generate_defensive_strength_fixtures(starting_gw, rolling):
    opposition_defensive_strength_fixtures = {}
    for i in api_team_names:
        for j in fixture_dict[i]:
            opposition_defensive_strength_fixtures[i] = calculate_opposition_defensive_strength_fixtures(fixture_dict[i])
    rolling_strength = {"team_name": [], "rolling_defensive_strength": []}
    for team_name, fixtures in opposition_defensive_strength_fixtures.items():
        rolling_strength["team_name"].append(team_name)
        fixtures_to_consider = fixtures[starting_gw-1:starting_gw-1+gws_to_roll]
        rolling_strength["rolling_defensive_strength"].append(int(np.mean([item[1] for item in fixtures_to_consider])))
    rolling_defensive_strength_df = pd.DataFrame(rolling_strength)
    rolling_defensive_strength_df.set_index("team_name", inplace=True)
    return rolling_defensive_strength_df


In [110]:
def generate_fixture_difficulty_df(starting_gw, rolling):
    rolling_attacking_strength_df = generate_attacking_strength_fixtures(starting_gw, rolling)
    rolling_defensive_strength_df = generate_defensive_strength_fixtures(starting_gw, rolling)
    return rolling_attacking_strength_df.join(rolling_defensive_strength_df)

OUTPUTS
- An measure of the attacking strength of the opposition
- An measure of the defensive strength of the opposition 

# Analysis

In [113]:
starting_gw = 1
gws_to_roll = 5

In [114]:
fixture_difficulty_df = generate_fixture_difficulty_df(starting_gw, 10)

In [115]:
fixture_difficulty_df.sort_values("rolling_defensive_strength")

,rolling_attacking_strength,rolling_defensive_strength
team_name,,
Brighton & Hove Albion FC,1094,1094
Brentford FC,1134,1094
Aston Villa FC,1110,1108
Watford FC,1128,1118
Everton FC,1110,1126
Liverpool FC,1104,1128
Leeds United FC,1150,1136
Manchester United FC,1108,1138
West Ham United FC,1128,1140


In [116]:
fixture_difficulty_df.sort_values("rolling_attacking_strength")

,rolling_attacking_strength,rolling_defensive_strength
team_name,,
Brighton & Hove Albion FC,1094,1094
Manchester City FC,1102,1160
Liverpool FC,1104,1128
Manchester United FC,1108,1138
Aston Villa FC,1110,1108
Everton FC,1110,1126
Wolverhampton Wanderers FC,1120,1158
West Ham United FC,1128,1140
Watford FC,1128,1118
